In [1]:
%matplotlib inline

In [2]:
from __future__ import print_function
import os

In [3]:
import ROOT
import h5py

In [4]:
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid.anchored_artists import AnchoredText

In [5]:
from uncertainties import ufloat, unumpy
from ToyMC.utils.Utils import is_logarithmic, get_bin_centers, get_bin_sizes

In [6]:
TYPE = 14
NAME = r'gevgen_{0}'.format(TYPE)
CACHE = '/data/icecube/data/GENIE_level0/nutev/{0}/cache.hdf5'.format(TYPE)
N_TARGET = 56
Y_BINNING = np.array([0, 0.001, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
#X_BINNING = np.logspace(-1.8, -0.10, 6)
#X_BINNING = np.linspace(0, 1, 6)
X_BINNING = np.array([0.0001, 0.03, 0.06, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
E_BINNING = np.array([30, 40, 50, 60, 70, 80, 90, 100, 120, 140, 160, 180, 200, 230, 260, 290, 320, 360])

In [7]:
is_cache = False
if os.path.isfile(CACHE):
    is_cache = True
print(is_cache)
if not is_cache:
    infile_nu = '/data/icecube/data/GENIE_level0/nutev/'+str(TYPE)+'/'+NAME+'.gst.root'
    infile_nubar = '/data/icecube/data/GENIE_level0/nutev/'+str(TYPE)+'/'+r'gevgen_-{0}.gst.root'.format(TYPE)
    input_file_nu = ROOT.TFile(infile_nu)
    input_file_nubar = ROOT.TFile(infile_nubar)
else:
    h5file = h5py.File(CACHE, 'r')
    for key in h5file.iterkeys():
        locals().update(dict([(key, np.array(h5file[key][:]))]))
    h5file.close()

False


In [8]:
def cache_events(in_arrays):
    h5file = h5py.File(CACHE, 'w')
    inarr_dict = dict(zip(STR_R, in_arrays))
    for key in inarr_dict.iterkeys():
        h5file.create_dataset(key, data=inarr_dict[key])
    h5file.close()

In [9]:
STR_R = ['pdg_array', 'xsec_array', 'energy_array', 'y_array', 'x_array', 'dis_array', 'weights']
if not is_cache:
    pdg_array, xsec_array, energy_array, y_array, x_array, dis_array, weights = [], [], [], [], [], [], []
    for event in input_file_nu.gst:
        pdg_array.append(event.neu)
        xsec_array.append(event.xsec * 1E-38)
        energy_array.append(event.Ev)
        y_array.append(event.y)
        x_array.append(event.x)
        dis_array.append(event.dis)
        weights.append(event.wght)
    for event in input_file_nubar.gst:
        pdg_array.append(event.neu)
        xsec_array.append(event.xsec * 1E-38)
        energy_array.append(event.Ev)
        y_array.append(event.y)
        x_array.append(event.x)
        dis_array.append(event.dis)
        weights.append(event.wght)
    pdg_array, xsec_array, energy_array, y_array, x_array, dis_array, weights = \
        map(np.array, (pdg_array, xsec_array, energy_array, y_array, x_array, dis_array, weights))
    cache_events((pdg_array, xsec_array, energy_array, y_array, x_array, dis_array, weights))
nu_mask = pdg_array > 0
nubar_mask = pdg_array < 0
print(xsec_array)
print(xsec_array.shape)
weights = np.ones(len(weights))
print(weights)

[  4.26148014e-36   3.20568848e-35   8.70505736e-36 ...,   3.86161249e-36
   6.23341528e-36   6.18216997e-37]
(2000000,)
[ 1.  1.  1. ...,  1.  1.  1.]


In [10]:
def format_axis(ax, x_label=r'', title=r'', lhs=False, ylim=None):
    ax.set_title(title)

    ax.set_xlim(np.min(Y_BINNING), np.max(Y_BINNING))
    for xmaj in ax.xaxis.get_majorticklocs():
        ax.axvline(x=xmaj, ls=':', color='gray', alpha=0.4, linewidth=1)
    if x_label == r'':
        ax.get_xaxis().set_ticks([])
    else:
        ax.set_xlabel(x_label, fontsize=14)
    if is_logarithmic(Y_BINNING):
        ax.set_xscale('log')

    ax.tick_params(axis='y', labelsize=10)
    if not ylim is None:
        ax.set_ylim(ylim)
    #ax.set_ylim(1, 7000)
    #ax.set_yscale('log')
    for ymaj in ax.yaxis.get_majorticklocs():
        ax.axhline(y=ymaj, ls=':', color='gray', alpha=0.4, linewidth=1)
    if not lhs:
        ax.get_yaxis().set_ticks([])
        xticks = ax.xaxis.get_major_ticks()
        try:
            xticks[0].set_visible(False)
        except:
            pass
    else:
        yticks = ax.yaxis.get_major_ticks()
        yticks[0].set_visible(False)

def plot_histo(ax, array, weights, xsec_weights, energy, x_bw, colour):
    #print('xsec_weights', xsec_weights)
    hist, edges = np.histogram(array, bins=Y_BINNING, weights=weights)
    hist_2, edges = np.histogram(array, bins=Y_BINNING, weights=weights**2)
    hist_w, edges = np.histogram(array, bins=Y_BINNING, weights=weights*xsec_weights)
    hist_w2, edges = np.histogram(array, bins=Y_BINNING, weights=(weights*xsec_weights)**2)

    u_hist = unumpy.uarray(hist, np.sqrt(hist_2))
    u_hist_w = unumpy.uarray(hist_w, np.sqrt(hist_w2))
    #dsigma = u_hist_w / u_hist
    dsigma, missing = [], []
    for idx in xrange(len(u_hist)):
        if u_hist[idx].n == 0:
            dsigma.append(ufloat(0, 0))
            missing.append(1)
            continue
        missing.append(0)
        dsigma.append(u_hist_w[idx] / u_hist[idx])
    dsigma, missing = np.array(dsigma), np.array(missing)

    b_sizes = get_bin_sizes(Y_BINNING)
    #print('b_sizes', b_sizes)
    #print('divide_factors', divide_factors)
    #print('b_sizes * divide_factors', b_sizes*divide_factors)
    dsigma_dxdy = (dsigma / (b_sizes.astype(float) * energy * x_bw)) / (N_TARGET * 1E-38)
    #print('dsigma_dxdy', unumpy.nominal_values(dsigma_dxdy))

    dsigma_dxdy_0 = np.concatenate(([dsigma_dxdy[0]], dsigma_dxdy))
    missing_0 = np.concatenate(([missing[0]], missing))
    ax.step(
        Y_BINNING, ma.masked_array(unumpy.nominal_values(dsigma_dxdy_0), mask=missing_0),
        alpha=0.5, color=colour, drawstyle='steps-mid', linewidth=2, linestyle=':'
    )

    ax.errorbar(
        Y_BINNING, ma.masked_array(unumpy.nominal_values(dsigma_dxdy_0), mask=missing_0),
        xerr=0, yerr=ma.masked_array(unumpy.std_devs(dsigma_dxdy_0), mask=missing_0),
        capsize=0.7, alpha=1, color=colour, linestyle='none', markersize=0.5, linewidth=1
    )
    
    return dsigma

In [11]:
e_bin_centers = get_bin_centers(E_BINNING)
x_bin_centers = get_bin_centers(X_BINNING)
e_bin_sizes = get_bin_sizes(E_BINNING)
x_bin_sizes = get_bin_sizes(X_BINNING)
E_PLOT = [65, 150, 245]
for e_idx, e_bin in enumerate(E_BINNING[:-1]):
    e_cont = True
    for x in E_PLOT:
        if x >= e_bin and x < E_BINNING[e_idx + 1]:
            e_cont = False
    if e_cont:
        continue
    print(r'E: {0}, {1}'.format(e_idx, e_bin_centers[e_idx]))
    energy_mask = (energy_array > e_bin) & (energy_array < E_BINNING[e_idx + 1])
    xsec_weight = xsec_array[energy_mask]
    y_array_e = y_array[energy_mask]
    x_array_e = x_array[energy_mask]
    weights_e = weights[energy_mask]
    nu_mask_e = nu_mask[energy_mask]
    nubar_mask_e = nubar_mask[energy_mask]

    fig = plt.figure(figsize=[12, 14])
    fig.suptitle('gevgen ID {0}'.format(TYPE), y=0.93)
    gs = gridspec.GridSpec(len(X_BINNING)-1, 1)
    gs.update(hspace=0., wspace=0.)
    
    dsigma_nu = dsigma_nubar = 0
    for x_idx, x_bin in enumerate(XS_BINNING[:-1]):
        #if x_idx == 1: break
        #print(r'     x: {0}, {1}'.format(x_idx, x_bin_centers[x_idx]))
        x_mask = (x_array_e > x_bin) & (x_array_e < X_BINNING[x_idx+1])
        y_array_e_x = y_array_e[x_mask]
        xsec_weight_x = xsec_weight[x_mask]
        weights_e_x = weights_e[x_mask]
        nu_mask_e_x = nu_mask_e[x_mask]
        nubar_mask_e_x = nubar_mask_e[x_mask]
        #print(r'     # events: {0}'.format(len(nu_mask_e_x)))
        
        gs0 = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs[x_idx],
                                               wspace=0, hspace=0, width_ratios=[1,1])
        ax0 = fig.add_subplot(gs0[0])
        dsigma_nu_x = plot_histo(
            ax0, y_array_e_x[nu_mask_e_x], weights_e_x[nu_mask_e_x],
            xsec_weight_x[nu_mask_e_x], energy=e_bin_centers[e_idx],
            x_bw=x_bin_sizes[x_idx], colour='red'
        )       
            
        ax1 = fig.add_subplot(gs0[1])
        dsigma_nubar_x = plot_histo(
            ax1, y_array_e_x[nubar_mask_e_x], weights_e_x[nubar_mask_e_x],
            xsec_weight_x[nubar_mask_e_x], energy=e_bin_centers[e_idx],
            x_bw=x_bin_sizes[x_idx], colour='blue'
        )

        #ylim = (np.min([ax0.get_ylim()[0], ax1.get_ylim()[0]]),
        #        np.max([ax0.get_ylim()[1], ax1.get_ylim()[1]]))
        ylim = (0, 100)
        ax0.set_ylim(ylim)
        ax1.set_ylim(ylim)
        if x_idx == 0:
            format_axis(ax0, title=r'Neutrino', lhs=True)
            format_axis(ax1, title=r'Anti-Neutrino')
        elif x_idx == len(X_BINNING)-2:
            format_axis(ax0, x_label=r'Y', lhs=True)
            format_axis(ax1, x_label=r'Y')
        else:
            format_axis(ax0, lhs=True)
            format_axis(ax1)
        
        at = AnchoredText(r'x = {0:.2f}'.format(x_bin_centers[x_idx]),
                          prop=dict(size=10), frameon=True, loc=1)
        at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
        ax1.add_artist(at)
        
        fig.text(0.055, 0.5, r'$\rm{1/E}$  $d^2\sigma/\rm{dx dy}$ $(\times 10^{-38}\rm{cm}^2/\rm{GeV})$ per nucleon',
                 rotation='vertical', va='center', size=20)
        fig.text(0.5, 0.075, r'(E = {0:.2f} GeV)'.format(e_bin_centers[e_idx]), ha='center', size=20)
        
        dsigma_nu += np.sum(dsigma_nu_x * x_bin_sizes[x_idx])
        dsigma_nubar += np.sum(dsigma_nubar_x * x_bin_sizes[x_idx])
        
        if x_idx == len(X_BINNING)-2:
            at0 = AnchoredText(r'd$\sigma_\nu$ / E = {0:.2E}'.format(dsigma_nu.n / e_bin_centers[e_idx])
                               +'\n'+r'ratio to WA = {0:.2f}'.format(
                    (dsigma_nu.n / e_bin_centers[e_idx]) / (0.677E-38 * N_TARGET)),
                               prop=dict(size=10), frameon=True, loc=2)
            at0.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
            ax0.add_artist(at0)
            
            at1 = AnchoredText(r'd$\sigma_\bar\nu$ / E = {0:.2E}'.format(dsigma_nubar.n / e_bin_centers[e_idx])
                               +'\n'+r'ratio to WA = {0:.2f}'.format(
                    (dsigma_nubar.n / e_bin_centers[e_idx]) / (0.334E-38 * N_TARGET)),
                               prop=dict(size=10), frameon=True, loc=2)
            at1.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
            ax1.add_artist(at1)
            
    print(r'd\sigma_nu / E = {0}:  WA = {1}'.format(dsigma_nu.n / e_bin_centers[e_idx], 0.677E-38 * N_TARGET))
    print(r'ratio {0}'.format((dsigma_nu.n / e_bin_centers[e_idx]) / (0.677E-38 * N_TARGET)))
    print(r'd\sigma_nubar / E = {0}:  WA = {1}'.format(dsigma_nubar.n / e_bin_centers[e_idx], 0.334E-38 * N_TARGET))
    print(r'ratio {0}'.format((dsigma_nubar.n / e_bin_centers[e_idx]) / (0.334E-38 * N_TARGET)))
        
    fig.savefig('./images/NuTeV/generated/'+NAME+'_'+'{0:.2f}_.png'.format(e_bin_centers[e_idx]),
                bbox_inches='tight')

E: 3, 64.8074069841


NameError: name 'XS_BINNING' is not defined

In [ ]:
print(e_bin_centers, x_bin_centers, e_bin_sizes, x_bin_sizes)